# 06 - Rule-Based Chatbot Systems

**Duration:** 2-3 hours | **Difficulty:** Intermediate

## 🎯 Learning Objectives
- Understand rule-based conversation systems
- Learn pattern matching and intent recognition
- Implement template-based response generation
- Build a complete rule-based chatbot

## 📚 Contents
1. Pattern Matching Fundamentals
2. Intent Recognition System
3. Response Generation Templates
4. Complete Chatbot Implementation
5. Analysis and Limitations

In [ ]:
import re
import random
import json
from typing import Dict, List, Tuple, Optional
from collections import defaultdict

# Import utilities
import sys
sys.path.append('../utils')
from chatbot_helpers import RuleBasedChatbot, ConversationContext
from text_utils import clean_text

print("Rule-based chatbot setup complete!")

## 1. Pattern Matching Fundamentals

Basic pattern matching for intent recognition.

In [ ]:
# Simple pattern matching example
def simple_chatbot(user_input: str) -> str:
    """
    Simple rule-based chatbot using pattern matching.
    """
    user_input = user_input.lower().strip()
    
    # Define patterns and responses
    patterns = {
        r'\b(hi|hello|hey)\b': [
            "Hello! How can I help you?",
            "Hi there! What can I do for you?"
        ],
        r'\bhow are you\b': [
            "I'm doing well, thank you!",
            "I'm great! How about you?"
        ],
        r'\bwhat.*name\b': [
            "I'm a helpful chatbot!",
            "You can call me ChatBot!"
        ],
        r'\b(bye|goodbye)\b': [
            "Goodbye! Have a great day!",
            "See you later!"
        ],
        r'\bthank.*you\b': [
            "You're welcome!",
            "Happy to help!"
        ]
    }
    
    # Match patterns
    for pattern, responses in patterns.items():
        if re.search(pattern, user_input):
            return random.choice(responses)
    
    return "That's interesting! Can you tell me more?"

# Test simple chatbot
test_inputs = [
    "Hello there!",
    "How are you?",
    "What's your name?",
    "Thank you!",
    "I like programming"
]

print("Simple Pattern Matching:")
for inp in test_inputs:
    response = simple_chatbot(inp)
    print(f"User: {inp}")
    print(f"Bot:  {response}\n")

## 2. Advanced Intent Recognition

More sophisticated pattern matching with confidence scores.

In [ ]:
class IntentClassifier:
    """
    Rule-based intent classifier with confidence scoring.
    """
    
    def __init__(self):
        self.intent_patterns = {
            'greeting': [
                r'\b(hi|hello|hey|good morning)\b',
                r'\bgreetings\b'
            ],
            'goodbye': [
                r'\b(bye|goodbye|see you)\b',
                r'\bgoodnight\b'
            ],
            'how_are_you': [
                r'\bhow are you\b',
                r'\bhow.*doing\b'
            ],
            'identity': [
                r'\bwhat.*name\b',
                r'\bwho are you\b'
            ],
            'help': [
                r'\bhelp\b',
                r'\bwhat can you do\b'
            ],
            'technical_question': [
                r'\b(machine learning|AI|programming)\b',
                r'\b(neural network|algorithm)\b'
            ],
            'gratitude': [
                r'\bthank.*you\b',
                r'\bthanks\b'
            ]
        }
    
    def classify_intent(self, text: str) -> Tuple[str, float]:
        """
        Classify intent with confidence score.
        """
        text_clean = clean_text(text, lowercase=True)
        intent_scores = {}
        
        for intent, patterns in self.intent_patterns.items():
            score = 0
            matches = 0
            
            for pattern in patterns:
                if re.search(pattern, text_clean):
                    matches += 1
                    score += len(pattern) / 100
            
            if matches > 0:
                intent_scores[intent] = score / len(patterns) + (matches / len(patterns))
        
        if intent_scores:
            best_intent = max(intent_scores, key=intent_scores.get)
            confidence = min(intent_scores[best_intent], 1.0)
            return best_intent, confidence
        
        return 'unknown', 0.0
    
    def extract_entities(self, text: str, intent: str) -> Dict[str, str]:
        """
        Extract entities based on intent.
        """
        entities = {}
        
        if intent == 'technical_question':
            topics = re.findall(r'\b(machine learning|AI|programming|neural network)\b', text.lower())
            if topics:
                entities['topic'] = topics[0]
        
        return entities

# Test intent classifier
classifier = IntentClassifier()

test_cases = [
    "Hello, how are you?",
    "Can you help with machine learning?",
    "What is your name?",
    "Thank you for your help!",
    "Goodbye!"
]

print("Intent Classification Results:")
for text in test_cases:
    intent, confidence = classifier.classify_intent(text)
    entities = classifier.extract_entities(text, intent)
    
    print(f"Text: '{text}'")
    print(f"Intent: {intent} (confidence: {confidence:.3f})")
    if entities:
        print(f"Entities: {entities}")
    print()

## 3. Response Generation System

Template-based response generation.

In [ ]:
class ResponseGenerator:
    """
    Generate responses based on intents and entities.
    """
    
    def __init__(self):
        self.responses = {
            'greeting': [
                "Hello! How can I help you today?",
                "Hi there! What can I do for you?",
                "Hey! Nice to see you!"
            ],
            'goodbye': [
                "Goodbye! Have a great day!",
                "See you later! Take care!",
                "Bye! Come back soon!"
            ],
            'how_are_you': [
                "I'm doing well, thank you! How are you?",
                "I'm great! How about you?",
                "Doing wonderful! How are you today?"
            ],
            'identity': [
                "I'm a helpful chatbot!",
                "You can call me ChatBot!",
                "I'm your AI assistant!"
            ],
            'help': [
                "I can answer questions and chat with you. What do you need?",
                "I'm here to help! What can I do for you?",
                "Feel free to ask me anything!"
            ],
            'technical_question': [
                "That's a great question about {topic}! What specifically interests you?",
                "I'd be happy to help with {topic}! What would you like to know?",
                "{topic} is fascinating! What aspect would you like to explore?"
            ],
            'gratitude': [
                "You're very welcome!",
                "Happy to help!",
                "My pleasure!"
            ],
            'unknown': [
                "That's interesting! Can you tell me more?",
                "I see. What else would you like to talk about?",
                "Thanks for sharing! What's on your mind?"
            ]
        }
    
    def generate(self, intent: str, entities: Dict[str, str] = None) -> str:
        """
        Generate response for given intent and entities.
        """
        if intent not in self.responses:
            intent = 'unknown'
        
        response = random.choice(self.responses[intent])
        
        # Fill in entity placeholders
        if entities:
            for key, value in entities.items():
                response = response.replace(f'{{{key}}}', value)
        
        return response

# Test response generator
generator = ResponseGenerator()

test_scenarios = [
    ('greeting', {}),
    ('technical_question', {'topic': 'machine learning'}),
    ('gratitude', {}),
    ('unknown', {})
]

print("Response Generation:")
for intent, entities in test_scenarios:
    response = generator.generate(intent, entities)
    print(f"Intent: {intent}, Entities: {entities}")
    print(f"Response: {response}\n")

## 4. Complete Chatbot Implementation

Integrating all components into a working chatbot.

In [ ]:
class RuleChatbot:
    """
    Complete rule-based chatbot system.
    """
    
    def __init__(self):
        self.classifier = IntentClassifier()
        self.generator = ResponseGenerator()
        self.context = ConversationContext(max_turns=5)
        self.turn_count = 0
    
    def chat(self, user_input: str) -> str:
        """
        Process user input and return response.
        """
        # Classify intent
        intent, confidence = self.classifier.classify_intent(user_input)
        
        # Extract entities
        entities = self.classifier.extract_entities(user_input, intent)
        
        # Generate response
        response = self.generator.generate(intent, entities)
        
        # Update context
        self.context.add_turn(user_input, response)
        self.turn_count += 1
        
        return response
    
    def get_stats(self) -> Dict[str, int]:
        """Get conversation statistics."""
        return {
            'turns': self.turn_count,
            'context_size': len(self.context.history)
        }

# Create and test chatbot
chatbot = RuleChatbot()

# Simulate conversation
conversation = [
    "Hello!",
    "What's your name?",
    "Can you help me with AI?",
    "Thank you!",
    "Goodbye!"
]

print("Complete Chatbot Demo:")
print("=" * 25)

for user_input in conversation:
    response = chatbot.chat(user_input)
    print(f"User: {user_input}")
    print(f"Bot:  {response}\n")

print(f"Statistics: {chatbot.get_stats()}")

## 5. Analysis and Limitations

Understanding rule-based chatbot strengths and weaknesses.

In [ ]:
# Analyze chatbot performance
def analyze_performance():
    """
    Analyze rule-based chatbot performance.
    """
    chatbot = RuleChatbot()
    
    # Test cases with expected intents
    test_cases = [
        ("Hello!", "greeting"),
        ("What is your name?", "identity"),
        ("Can you help with AI?", "technical_question"),
        ("Thank you", "gratitude"),
        ("Goodbye", "goodbye"),
        ("I like pizza", "unknown")
    ]
    
    print("Performance Analysis:")
    print("=" * 20)
    
    correct = 0
    for user_input, expected in test_cases:
        predicted, confidence = chatbot.classifier.classify_intent(user_input)
        is_correct = predicted == expected
        
        if is_correct:
            correct += 1
        
        status = "✓" if is_correct else "✗"
        print(f"{status} '{user_input}' -> {predicted} ({confidence:.3f})")
    
    accuracy = correct / len(test_cases)
    print(f"\nAccuracy: {accuracy:.2%}")
    
    print("\nSTRENGTHS:")
    print("• Fast and predictable")
    print("• No training data needed")
    print("• Easy to understand")
    print("• Good for specific domains")
    
    print("\nLIMITATIONS:")
    print("• Cannot handle language variations")
    print("• Requires manual pattern creation")
    print("• Limited contextual understanding")
    print("• Doesn't learn from interactions")

analyze_performance()

## 🎉 Congratulations!

You've built a complete rule-based chatbot system:

✅ **Pattern Matching**: Regular expressions for intent recognition  
✅ **Intent Classification**: Confidence-based intent detection  
✅ **Response Generation**: Template-based response system  
✅ **Complete System**: Integrated chatbot with conversation tracking  
✅ **Analysis**: Understanding strengths and limitations  

## 🚀 Next Steps

In the next notebook, we'll explore retrieval-based chatbots:
- Information retrieval techniques
- Embedding-based similarity
- Response ranking systems

**Ready for retrieval systems?** Continue to [`07_retrieval_based_chatbot.ipynb`](07_retrieval_based_chatbot.ipynb)!